### Read the script

In [9]:
import os
from ieeg.io import get_data, raw_from_layout, save_derivative

In [ ]:
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab")
layout = get_data("LexicalDecRepDelay", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")

In [ ]:
#for subject in subjects:
#    raw = raw_from_layout(layout, subject=subject, preload=True, extension='.edf')
    # do stuff with each subject data

subject = subjects[0]
raw = raw_from_layout(layout, subject=subject, preload=True, extension='.edf')
print(subjects)

In [ ]:
raw.plot()

### Linear noise filtering
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_clean.html

In [6]:
from ieeg.mt_filter import line_filter
from ieeg.viz.ensemble import figure_compare

In [ ]:
# n_jobs=-1 means take all the processors, n_jobs=-2 means take all - 1 processors
# Keep all the other parameters not changed
# %% filter data
line_filter(raw, mt_bandwidth=10., n_jobs=-3, copy=False, verbose=10,
            filter_length='700ms', freqs=[60], notch_widths=20)
line_filter(raw, mt_bandwidth=10., n_jobs=-3, copy=False, verbose=10,
            filter_length='20s', freqs=[60, 120, 180, 240],
            notch_widths=20)

In [ ]:
# Check if derivatives folder exists and create if not
bids_root='C:\\Users\\bl314\\Box\\CoganLab\\BIDS-1.0_LexicalDecRepDelay\\BIDS'
if not os.path.exists(os.path.join(bids_root, "derivatives")):
    os.mkdir(os.path.join(bids_root, "derivatives"))
    os.mkdir(os.path.join(bids_root, "derivatives", "clean"))
elif not os.path.exists(os.path.join(bids_root, "derivatives", "clean")):
    os.mkdir(os.path.join(bids_root, "derivatives", "clean"))
save_derivative(raw, layout, "clean", True)

### Remove bad channels
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_spectrograms.html


In [13]:
from ieeg.viz.ensemble import chan_grid
from ieeg.timefreq.multitaper import spectrogram
from ieeg.timefreq.utils import crop_pad
from ieeg.navigate import channel_outlier_marker
from ieeg.viz.parula import parula_map
import numpy as np
import mne

In [14]:
# mark channel outliers as bad
raw.info['bads'] = channel_outlier_marker(raw, 3)

outlier round 1 channels: ['RFOA18']
outlier round 1 channels: ['RFOA18', 'RTMM11']
outlier round 2 channels: ['RFOA18', 'RTMM11', 'RFAM4']
outlier round 2 channels: ['RFOA18', 'RTMM11', 'RFAM4', 'RTAM13']
outlier round 2 channels: ['RFOA18', 'RTMM11', 'RFAM4', 'RTAM13', 'RIP10']
outlier round 3 channels: ['RFOA18', 'RTMM11', 'RFAM4', 'RTAM13', 'RIP10', 'RFOA2']
outlier round 3 channels: ['RFOA18', 'RTMM11', 'RFAM4', 'RTAM13', 'RIP10', 'RFOA2', 'RFOA3']
outlier round 3 channels: ['RFOA18', 'RTMM11', 'RFAM4', 'RTAM13', 'RIP10', 'RFOA2', 'RFOA3', 'RFOP1']
outlier round 3 channels: ['RFOA18', 'RTMM11', 'RFAM4', 'RTAM13', 'RIP10', 'RFOA2', 'RFOA3', 'RFOP1', 'RTMM9']
outlier round 4 channels: ['RFOA18', 'RTMM11', 'RFAM4', 'RTAM13', 'RIP10', 'RFOA2', 'RFOA3', 'RFOP1', 'RTMM9', 'RFOA7']
outlier round 5 channels: ['RFOA18', 'RTMM11', 'RFAM4', 'RTAM13', 'RIP10', 'RFOA2', 'RFOA3', 'RFOP1', 'RTMM9', 'RFOA7', 'RFOA10']
outlier round 6 channels: ['RFOA18', 'RTMM11', 'RFAM4', 'RTAM13', 'RIP10', 'RFO

In [15]:
# Exclude bad channels
raw.drop_channels(raw.info['bads'])
raw.load_data()

<RawEDF | sub-D0101_task-LexicalDecRepDelay_acq-01_run-01_ieeg.edf, 221 x 8134656 (3972.0 s), ~13.39 GB, data loaded>

In [16]:
# CAR (common average reference)
raw.set_eeg_reference(ref_channels="average", ch_type='seeg')

Applying average reference.
Applying a custom ('sEEG',) reference.


<RawEDF | sub-D0101_task-LexicalDecRepDelay_acq-01_run-01_ieeg.edf, 221 x 8134656 (3972.0 s), ~13.39 GB, data loaded>

In [17]:
raw.plot()

### Remove bad trials
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_spectrograms_wavelet.html

In [18]:
# Mark channel outliers as bad
raw.info['bads'] = channel_outlier_marker(raw, 5)

In [19]:
raw = raw.drop_channels(raw.info['bads'])

### Multitaper spectrogram plot

#### Calculate Spectra

In [ ]:
freq = np.arange(10, 200., 6.)
spectra = spectrogram(raw, freq, 'CORRECT', -1.2, 1.2, 'Yes/No', -1.2, 0.2,
                      n_jobs=1, verbose=10, time_bandwidth=10, n_cycles=freq/2)
crop_pad(spectra, "0.5s")

#### Plot data

In [ ]:
chan_grid(spectra, vlim=(0, 2), cmap=parula_map)